In [ ]:
import os
import logging
import numpy as np
import rasterio
from rasterio import windows
from rasterio.warp import calculate_default_transform, reproject, Resampling
from shapely.geometry import box, Point
import geopandas as gpd
from tqdm import tqdm

def reproject_raster(src_path, dst_path, dst_crs):
    """
    Reproject a raster to a specified CRS.

    Parameters:
    - src_path (str): Path to source raster
    - dst_path (str): Path to save reprojected raster
    - dst_crs (str or rasterio.crs.CRS): Destination Coordinate Reference System

    Returns:
    - dict: Metadata of the reprojected raster
    """
    with rasterio.open(src_path) as src:
        # Applying transformation parameters for reprojection
        transform, width, height = calculate_default_transform(
            src.crs, dst_crs, src.width, src.height,
            *src.bounds
        )

        # Prepare metadata for the destination raster
        kwargs = src.meta.copy()
        kwargs.update({
            'crs': dst_crs,
            'transform': transform,
            'width': width,
            'height': height
        })

        # Prepare the destination raster
        with rasterio.open(dst_path, 'w', **kwargs) as dst:
            # Reprojecting the raster to the specified CRS
            for i in tqdm(range(1, src.count + 1), desc=f"Reprojecting bands of {os.path.basename(src_path)}"):
                src_array = src.read(i)
                dst_array = np.empty((height, width), dtype=src_array.dtype)

                reproject(
                    source=src_array,
                    destination=dst_array,
                    src_transform=src.transform,
                    src_crs=src.crs,
                    dst_transform=transform,
                    dst_crs=dst_crs,
                    resampling=Resampling.nearest
                )

                dst.write(dst_array, i)

        return kwargs

def clip_points_from_rasters(
    geojson_file,
    raster_folder,
    output_dir="cropped_outputs",
    pixel_buffer=7,
    reprojected_raster_dir=None
):
    """
    Clip raster images around specific points from a GeoJSON file,
    with automatic raster reprojection and progress tracking.

    Parameters:
    - geojson_file (str): Path to the input GeoJSON file containing points
    - raster_folder (str): Directory containing input raster files
    - output_dir (str, optional): Directory to save clipped raster images.
      Defaults to "cropped_outputs"
    - pixel_buffer (int, optional): Number of pixels to buffer around the point.
      Defaults to 7
    - reprojected_raster_dir (str, optional): Directory to save reprojected rasters.
      If None, uses a subdirectory in output_dir
    """
    # Configure logging
    logging.basicConfig(level=logging.INFO,
                        format='%(asctime)s - %(levelname)s - %(message)s')
    logger = logging.getLogger(__name__)

    # Create output directories
    os.makedirs(output_dir, exist_ok=True)

    # Reprojecting the raster to the specified CRS
    if reprojected_raster_dir is None:
        reprojected_raster_dir = os.path.join(output_dir, "reprojected_rasters")
    os.makedirs(reprojected_raster_dir, exist_ok=True)

    # Read the GeoJSON file
    try:
        gdf = gpd.read_file(geojson_file)
    except Exception as e:
        logger.error(f"Error reading GeoJSON file: {e}")
        raise

    # Ensure the GeoJSON has a defined Coordinate Reference System (CRS)
    if not gdf.crs:
        logger.warning("GeoJSON has no CRS. Assuming EPSG:4326 (WGS84).")
        gdf = gdf.set_crs('EPSG:4326')

    # Get the CRS of the GeoJSON
    geojson_crs = gdf.crs.to_string()
    logger.info(f"GeoJSON CRS: {geojson_crs}")

    # Reprojecting the raster to the specified CRS
    gdf_proj = gdf.to_crs(geojson_crs)

    # Track successfully processed and skipped points
    total_points = len(gdf_proj)
    processed_points = 0
    skipped_points = 0

    # Get list of TIFF files and wrap with tqdm for progress tracking
    tiff_files = [f for f in os.listdir(raster_folder) if f.endswith(".tif")]

    # Iterate through raster files with progress bar
    for raster_file in tqdm(tiff_files, desc="Processing Rasters"):
        # Construct full path to the raster file
        raster_path = os.path.join(raster_folder, raster_file)

        # Open the raster file
        try:
            with rasterio.open(raster_path) as src:
                # Reprojecting the raster to the specified CRS
                current_raster_crs = src.crs.to_string()
                logger.info(f"Raster {raster_file} CRS: {current_raster_crs}")

                # Reprojecting the raster to the specified CRS
                if current_raster_crs != geojson_crs:
                    # Reprojecting the raster to the specified CRS
                    reprojected_raster_path = os.path.join(
                        reprojected_raster_dir,
                        f"reprojected_{raster_file}"
                    )
                    logger.info(f"Reprojecting {raster_file} to {geojson_crs}")

                    try:
                        reproject_raster(
                            raster_path,
                            reprojected_raster_path,
                            geojson_crs
                        )
                        raster_path = reprojected_raster_path
                    except Exception as e:
                        logger.error(f"Failed to reproject {raster_file}: {e}")
                        continue

                # Reprojecting the raster to the specified CRS
                with rasterio.open(raster_path) as src:
                    # Create output subdirectory for this raster
                    output_subdir = os.path.join(output_dir, os.path.splitext(raster_file)[0])
                    os.makedirs(output_subdir, exist_ok=True)

                    # Create bounding box of the raster
                    bounds_geom = box(*src.bounds)

                    # Reprojecting the raster to the specified CRS
                    for idx, row in tqdm(gdf_proj.iterrows(),
                                         total=len(gdf_proj),
                                         desc=f"Extracting points from {raster_file}",
                                         leave=False):
                        point = row.geometry

                        # Skip if not a point or outside raster bounds
                        if not isinstance(point, Point) or not bounds_geom.contains(point):
                            skipped_points += 1
                            continue

                        # Convert geographic coordinates to pixel coordinates
                        try:
                            px, py = ~src.transform * (point.x, point.y)
                            px, py = int(px), int(py)
                        except Exception as e:
                            logger.warning(f"Could not convert point {idx} to pixel coordinates: {e}")
                            skipped_points += 1
                            continue

                        # Define window around the point
                        window = windows.Window(
                            px - pixel_buffer, py - pixel_buffer,
                            2 * pixel_buffer + 1, 2 * pixel_buffer + 1
                        )

                        # Skip if window is outside raster bounds
                        if (
                            window.col_off < 0 or window.row_off < 0 or
                            window.col_off + window.width > src.width or
                            window.row_off + window.height > src.height
                        ):
                            skipped_points += 1
                            continue

                        # Read data for the window
                        data = src.read(window=window)

                        # Applying transformation parameters for reprojection
                        crop_transform = windows.transform(window, src.transform)

                        # Create output path for this crop
                        crop_path = os.path.join(output_subdir, f"{idx}.tif")

                        # Writing the output data
                        try:
                            with rasterio.open(
                                crop_path, 'w',
                                driver='GTiff',
                                height=data.shape[1],
                                width=data.shape[2],
                                count=data.shape[0],
                                dtype=data.dtype,
                                crs=src.crs,
                                transform=crop_transform
                            ) as dst:
                                dst.write(data)
                            processed_points += 1
                        except Exception as e:
                            logger.error(f"Error writing crop for point {idx}: {e}")

        except Exception as e:
            logger.error(f"Error processing raster {raster_file}: {e}")

    # Log summary
    logger.info(f"Total points in GeoJSON: {total_points}")
    logger.info(f"Processed points: {processed_points}")
    logger.info(f"Skipped points: {skipped_points}")

# Example usage
if __name__ == "__main__":
    clip_points_from_rasters(
        geojson_file= r"C:\Aarde, Economie & Duurzaamheid\Scriptie Uitbreiding\Dataset_Malaga.geojson",
        raster_folder= r"C:\SatelliteImagery",
        output_dir= r"C:\SatelliteImagery\clipped_outputs",
        pixel_buffer= 7,  # This gives 15x15 crops (2*7 + 1)
        reprojected_raster_dir=None  # Will use a subdirectory in output_dir
    )

2025-05-12 17:14:04,202 - INFO - GeoJSON CRS: EPSG:3035
Processing Rasters:   0%|                                                                        | 0/1 [00:00<?, ?it/s]2025-05-12 17:14:04,442 - INFO - Raster Test1.tif CRS: EPSG:32630
2025-05-12 17:14:04,458 - INFO - Reprojecting Test1.tif to EPSG:3035

Reprojecting bands of Test1.tif: 100%|███████████████████████████████████████████████████| 8/8 [10:31<00:00, 78.93s/it]

Processing Rasters: 100%|███████████████████████████████████████████████████████████████| 1/1 [12:02<00:00, 722.93s/it]
2025-05-12 17:26:07,199 - INFO - Total points in GeoJSON: 16114
2025-05-12 17:26:07,208 - INFO - Processed points: 756
2025-05-12 17:26:07,208 - INFO - Skipped points: 15358
